In [1]:
import mne
import librosa
import numpy as np
import pandas as pd
import os
from colorama import Fore, Style

class DataPreprocessing:
    def __init__(self):
        self.timestamps_df = None

    @staticmethod
    def compute_cepstrum_mel(data, sfreq, n_mfcc=20):
        #stft = librosa.stft(data)
        #power_spectrum = np.abs(stft) ** 2
        mfccs = librosa.feature.mfcc(y=data, sr=sfreq, n_mfcc=n_mfcc)
        return mfccs
    
    def print_decorative_log(self, message, color=Fore.BLUE, style=Style.RESET_ALL):
        line_length = len(message) + 4  # Length of the message plus padding on both sides
        decorative_line = "#" * line_length
        print(color + decorative_line)
        print(f"# {message} #")
        print(decorative_line + style)

    def load_timestamps(self, csv_file_path):
        self.timestamps_df = pd.read_csv(csv_file_path)

    def get_timestamps(self, file_name):
        file_name = os.path.splitext(file_name)[0]
        patient_id = file_name
        timestamps = self.timestamps_df[self.timestamps_df['Sub File'] == patient_id]
        return timestamps.iloc[0]

    def preprocess_data(self, edf_file_path, channel_pairs, channel_pairs_joined, file_name, n_mfcc=20):
        patient_id = os.path.splitext(file_name)[0]
        self.print_decorative_log("Starting Preprocessing Sequence", Fore.GREEN)
        self.print_decorative_log("Patient"+patient_id, Fore.YELLOW)

        # Load the EDF file
        raw = mne.io.read_raw_edf(edf_file_path, preload=True)
        self.print_decorative_log(("Patient" + patient_id + "EDF Loaded ... "), Fore.YELLOW)
        
        desired_channels = ['EEG Fp1', 'EEG F3', 'EEG C3', 'EEG P3', 'EEG O1', 'EEG F7', 'EEG T3', 'EEG T5', 'EEG Fc1', 'EEG Fc5', 'EEG Cp1', 'EEG Cp5', 'EEG F9', 'EEG Fz', 'EEG Cz', 'EEG Pz', 'EEG Fp2', 'EEG F4', 'EEG C4', 'EEG P4', 'EEG O2', 'EEG F8', 'EEG T4', 'EEG T6', 'EEG Fc2', 'EEG Fc6', 'EEG Cp2', 'EEG Cp6', 'EEG F10', 'EKG EKG', 'SPO2', 'HR', '1', '2', 'MK']
        #Drop extra channels
        # Check the number of channels
        if len(raw.ch_names) > 35:
            for i, channel_name in enumerate(raw.ch_names):
                if 'EEG FP2' in channel_name:
                    raw.rename_channels({channel_name: 'Fp2'})
            # Drop channels not found in the desired channel list
            channels_to_drop = [channel_name for channel_name in raw.ch_names if channel_name not in desired_channels]
            raw.drop_channels(channels_to_drop)
            self.print_decorative_log("Extra Channels Dropped ... ", Fore.RED)          
        # Reorder the channels to match the standard ordering for the dataset
        channels_order = desired_channels
        #Rename the channels to match the standard naming for the dataset
        raw.rename_channels(dict(zip(raw.ch_names, channels_order)))
        # Reorder channels
        raw = raw.pick_channels(channels_order)
        # Change the channel type for a specific channel
        channel_names = ['2', 'SPO2', 'HR', 'MK']
        new_channel_type = ['ecg', 'hbo', 'hbo', 'hbo']
        raw.set_channel_types(dict(zip(channel_names, new_channel_type)))
        
        self.print_decorative_log("Channel Changed ... ", Fore.YELLOW)

        # Filtering to remove slow drifts
        filt_raw = raw.copy().filter(l_freq=1.0, h_freq=None)
        self.print_decorative_log("Slow drifts removed ... ", Fore.YELLOW)

        # Apply ICA to remove ECG artifacts

        ica = mne.preprocessing.ICA(n_components=15, max_iter="auto", random_state=97)
        ica.fit(filt_raw)
        ica.exclude = []
        ecg_indices, ecg_scores = ica.find_bads_ecg(raw, method="correlation", threshold="auto")
        ica.exclude = ecg_indices

        reconst_raw = raw.copy()
        ica.apply(reconst_raw)

        self.print_decorative_log("ECG Artificats Removed... ", Fore.YELLOW)


        # Perform bipolar longitudinal referencing
        anodes = []
        cathodes = []
        for pair in channel_pairs:
            anodes.append(pair[0])
            cathodes.append(pair[1])

        raw_bip_ref = mne.set_bipolar_reference(reconst_raw, anode=anodes, cathode=cathodes)
        raw_bip_ref_ch = raw_bip_ref.copy().pick_channels(channel_pairs_joined)
        self.print_decorative_log("Bipolar Referencing Done ... ", Fore.YELLOW)
        raw_clean = mne.preprocessing.oversampled_temporal_projection(raw_bip_ref_ch)
        raw_clean.filter(0.0, 40.0)
        self.print_decorative_log("Smoothing & Filtering Done ... ", Fore.YELLOW)

        # Extract the timestamps
        timestamps = self.get_timestamps(file_name)
        pre_ictal_start = timestamps['Pre Ictal Start Time']
        pre_ictal_end = timestamps['Pre Ictal End Time']
        ictal_start = timestamps['Ictal Start Time']
        ictal_end = timestamps['Ictal End Time']
        post_ictal_start = timestamps['Post Ictal Start Time']
        post_ictal_end = timestamps['Post Ictal End Time']
        self.print_decorative_log("Timestamps Extracted ... ", Fore.YELLOW)

        # Extract the data
        pre_ictal_data = raw_clean.get_data(picks=channel_pairs_joined, tmin=pre_ictal_start, tmax=pre_ictal_end)
        ictal_data = raw_clean.get_data(picks=channel_pairs_joined, tmin=ictal_start, tmax=ictal_end)
        post_ictal_data = raw_clean.get_data(picks=channel_pairs_joined, tmin=post_ictal_start, tmax=post_ictal_end)
        normal_data1 = raw_clean.get_data(picks=channel_pairs_joined, tmax=pre_ictal_start)
        normal_data2 = raw_clean.get_data(picks=channel_pairs_joined, tmin=post_ictal_end)
        normal_data = np.append(normal_data1, normal_data2, axis=1)
        
        # Extract the cepstral features
        sfreq = raw.info['sfreq']
        cepstrum_mel_features = []
        cepstral_features = []
        for channel_data in pre_ictal_data:
            cepstrum_mel = self.compute_cepstrum_mel(channel_data, sfreq, n_mfcc)
            cepstrum_mel_features.append(cepstrum_mel)
        self.print_decorative_log("Pre-Ictal Feature Extracted ... ", Fore.YELLOW)
        
        for channel_data in ictal_data:
            cepstrum_mel = self.compute_cepstrum_mel(channel_data, sfreq, n_mfcc)
            cepstrum_mel_features.append(cepstrum_mel)
        self.print_decorative_log("Ictal Feature Extracted ... ", Fore.YELLOW)

        for channel_data in post_ictal_data:
            cepstrum_mel = self.compute_cepstrum_mel(channel_data, sfreq, n_mfcc)
            cepstrum_mel_features.append(cepstrum_mel)
        self.print_decorative_log("Post-Ictal Feature Extracted ... ", Fore.YELLOW)

        for channel_data in normal_data:
            cepstrum_mel = self.compute_cepstrum_mel(channel_data, sfreq, n_mfcc)
            cepstrum_mel_features.append(cepstrum_mel)
        self.print_decorative_log("Normal Feature Extracted ... ", Fore.YELLOW)

        cepstral_features.append(cepstrum_mel_features)
        self.print_decorative_log("Cepstral Features Extracted for"+ patient_id +"... ", Fore.GREEN)

        return cepstral_features
    
    def preprocess_patient_data(self, patient_folder_path, channel_pairs, channel_pairs_joined, n_mfcc=20):
        file_names = [file for file in os.listdir(patient_folder_path) if file.endswith('.edf')]
        features = []
        labels = []
        for file_name in file_names:
            patient_id = os.path.splitext(file_name)[0]
            edf_file_path = os.path.join(patient_folder_path, file_name)
            patient__features = self.preprocess_data(edf_file_path, channel_pairs, channel_pairs_joined, file_name, n_mfcc)
            
            features.append(patient__features)
            labels.extend([0, 1, 2, 3])  # 0: Pre-Ictal, 1: Ictal, 2: Post-Ictal, 3: Normal

        self.print_decorative_log("Patient:" + patient_id + "Preprocessing Completed ... ", Fore.GREEN)
        return features, labels

    def preprocess_all_patients(self, main_folder_path, channel_pairs, channel_pairs_joined, n_mfcc=20):
        patient_folders = os.listdir(main_folder_path)
        all_features = []
        all_labels = []

        for patient_folder in patient_folders:
            patient_folder_path = os.path.join(main_folder_path, patient_folder)
            if os.path.isdir(patient_folder_path):
                features, labels = self.preprocess_patient_data(patient_folder_path, channel_pairs, channel_pairs_joined, n_mfcc)
                all_features.extend(features)
                all_labels.extend(labels)
        self.print_decorative_log("All Patients Preprocessing Completed ... ", Fore.GREEN)
        return all_features, all_labels

    def export_to_csv(self, features, labels, output_file_path):
        data = np.concatenate(features, axis=0)
        label_column = np.array(labels).reshape(-1, 1)
        data_with_labels = np.concatenate((data, label_column), axis=1)
        df = pd.DataFrame(data_with_labels)
        df.to_csv(output_file_path, index=False, header=False)
        self.print_decorative_log(f"Data Exported to" + output_file_path + "... ", Fore.GREEN)

In [2]:
# Define the channel pairs 
channel_pairs = [['EEG Fp1', 'EEG F7'], ['EEG F7', 'EEG T3'], ['EEG T3', 'EEG T5'], ['EEG T5', 'EEG O1'],
                 ['EEG Fp1', 'EEG F3'], ['EEG C3', 'EEG F3'], ['EEG F3', 'EEG O1'], ['EEG Fp2', 'EEG F4'],
                 ['EEG F4', 'EEG C4'], ['EEG C4', 'EEG P4'], ['EEG P4', 'EEG O2'], ['EEG Fp2', 'EEG F8'],
                 ['EEG F8', 'EEG T4'], ['EEG T4', 'EEG T6'], ['EEG T6', 'EEG O2']]
channel_pairs_joined = ['{}-{}'.format(pair[0], pair[1]) for pair in channel_pairs]

data_preprocessing = DataPreprocessing()
data_preprocessing.load_timestamps('Siena-DB-timestamp.csv')

main_folder_path = 'Siena-Scalp-EEG-Database-v1.0.0\siena-scalp-eeg-database-1.0.0'
all_features, all_labels = data_preprocessing.preprocess_all_patients(main_folder_path, channel_pairs, channel_pairs_joined, n_mfcc=20)
data_preprocessing.export_to_csv(all_features, all_labels, 'merged_dataset.csv')

###################################
# Starting Preprocessing Sequence #
###################################
#################
# PatientPN00-1 #
#################
Extracting EDF parameters from c:\Data\Epsilon-CDSP\Final-Project\NeuroGuard\Siena-Scalp-EEG-Database-v1.0.0\siena-scalp-eeg-database-1.0.0\PN00\PN00-1.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 1343999  =      0.000 ...  2624.998 secs...
################################
# PatientPN00-1EDF Loaded ...  #
################################
NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).
########################
# Channel Changed ...  #
########################
Filtering raw data in 1 contiguous segment
Setting up high-pass filter at 1 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal highpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 

C:\Users\MUSA\AppData\Local\Temp\ipykernel_26772\3509405346.py:60: RuntimeWarning: The unit for channel(s) HR, MK, SPO2 has changed from V to M.
  raw.set_channel_types(dict(zip(channel_names, new_channel_type)))
[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s


############################
# Slow drifts removed ...  #
############################
Fitting ICA to data using 34 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 24.6s.
... filtering ICA sources
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5120 samples (10.000 s)

... filtering target
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Low

c:\Users\MUSA\anaconda3\envs\NeuroGuard\Lib\site-packages\mne\preprocessing\otp.py:24: RuntimeWarning: invalid value encountered in sqrt
  s = np.sqrt(s, out=s)


    Denoising   180.00 –   190.00 s
    Denoising   190.00 –   200.00 s
    Denoising   200.00 –   210.00 s
    Denoising   210.00 –   220.00 s
    Denoising   220.00 –   230.00 s
    Denoising   230.00 –   240.00 s
    Denoising   240.00 –   250.00 s
    Denoising   250.00 –   260.00 s
    Denoising   260.00 –   270.00 s
    Denoising   270.00 –   280.00 s
    Denoising   280.00 –   290.00 s
    Denoising   290.00 –   300.00 s
    Denoising   300.00 –   310.00 s
    Denoising   310.00 –   320.00 s
    Denoising   320.00 –   330.00 s
    Denoising   330.00 –   340.00 s
    Denoising   340.00 –   350.00 s
    Denoising   350.00 –   360.00 s
    Denoising   360.00 –   370.00 s
    Denoising   370.00 –   380.00 s
    Denoising   380.00 –   390.00 s
    Denoising   390.00 –   400.00 s
    Denoising   400.00 –   410.00 s
    Denoising   410.00 –   420.00 s
    Denoising   420.00 –   430.00 s
    Denoising   430.00 –   440.00 s
    Denoising   440.00 –   450.00 s
    Denoising   450.00 –   4

...

[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    2.1s


############################
# Slow drifts removed ...  #
############################
Fitting ICA to data using 34 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 56.2s.
... filtering ICA sources
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 8.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 7.75 Hz)
- Upper passband edge: 16.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 16.25 Hz)
- Filter length: 5120 samples (10.000 s)

... filtering target
Setting up band-pass filter from 8 - 16 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Low

c:\Users\MUSA\anaconda3\envs\NeuroGuard\Lib\site-packages\mne\preprocessing\otp.py:24: RuntimeWarning: invalid value encountered in sqrt
  s = np.sqrt(s, out=s)


    Denoising    70.00 –    80.00 s
    Denoising    80.00 –    90.00 s
    Denoising    90.00 –   100.00 s
    Denoising   100.00 –   110.00 s
    Denoising   110.00 –   120.00 s
    Denoising   120.00 –   130.00 s
    Denoising   130.00 –   140.00 s
    Denoising   140.00 –   150.00 s
    Denoising   150.00 –   160.00 s
    Denoising   160.00 –   170.00 s
    Denoising   170.00 –   180.00 s
    Denoising   180.00 –   190.00 s
    Denoising   190.00 –   200.00 s
    Denoising   200.00 –   210.00 s
    Denoising   210.00 –   220.00 s
    Denoising   220.00 –   230.00 s
    Denoising   230.00 –   240.00 s
    Denoising   240.00 –   250.00 s
    Denoising   250.00 –   260.00 s
    Denoising   260.00 –   270.00 s
    Denoising   270.00 –   280.00 s
    Denoising   280.00 –   290.00 s
    Denoising   290.00 –   300.00 s
    Denoising   300.00 –   310.00 s
    Denoising   310.00 –   320.00 s
    Denoising   320.00 –   330.00 s
    Denoising   330.00 –   340.00 s
    Denoising   340.00 –   3

ValueError: New channel names are not unique, renaming failed